In [ ]:
import os
import zipfile
from pathlib import Path
import datetime

# Configuration
source_directory = "."
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f"deeplearning_ai_course_backup_{timestamp}.zip"

# Files and directories to exclude
exclude_patterns = {'.git', '__pycache__', '.DS_Store', '.ipynb_checkpoints', 'node_modules', '.env'}

def should_exclude(file_path):
    """Check if a file should be excluded."""
    path_parts = Path(file_path).parts
    return any(pattern in path_parts for pattern in exclude_patterns)

def get_all_files(directory):
    """Get all files in directory and subdirectories."""
    all_files = []
    for root, dirs, files in os.walk(directory):
        dirs[:] = [d for d in dirs if not should_exclude(os.path.join(root, d))]
        for file in files:
            file_path = os.path.join(root, file)
            if not should_exclude(file_path):
                all_files.append(file_path)
    return all_files

# Get all files and create zip
print("Scanning for files...")
files_to_zip = get_all_files(source_directory)
print(f"Found {len(files_to_zip)} files to zip")

# Calculate total size
total_size = sum(os.path.getsize(f) for f in files_to_zip if os.path.exists(f))
print(f"Total size: {total_size / (1024*1024):.2f} MB")

# Create zip file
print(f"Creating zip file: {zip_filename}")
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED, compresslevel=6) as zipf:
    for file_path in files_to_zip:
        try:
            arcname = os.path.relpath(file_path, source_directory)
            zipf.write(file_path, arcname)
        except Exception as e:
            print(f"Warning: Could not zip {file_path}: {e}")

# Show results
if os.path.exists(zip_filename):
    zip_size = os.path.getsize(zip_filename)
    print(f"\n✓ Zip file created: {zip_filename}")
    print(f"✓ Zip size: {zip_size / (1024*1024):.2f} MB")
    print(f"✓ Compression: {((total_size - zip_size) / total_size * 100):.1f}%")
    print("\nBackup complete!")
else:
    print("❌ Failed to create zip file")
